# Majorana Tapering 
### in the Stabilizer Subspace Projection formalism
Here, we take a look at the qubit reduction technique of [tapering](https://arxiv.org/abs/1701.08213) and an implementation based on the core `S3_projection` class. Unlike [Contextual-Subspace VQE](https://doi.org/10.22331/q-2021-05-14-456), this technique is *exact*, in the sense that it perfectly preserves the energy spectrum of the input operator.

At the core of qubit tapering is a symmetry of the Hamiltonian, which in this case means a set of universally commuting operators. The idea is that these operators must be simultaneously measureable and so can be treated independently of the remaining Hamiltonian terms. The method works by finding an independent generating set for the symmetry and seeks to find the 'correct' assignment of eigenvalues (called a *sector*), which completely determines the measurement outcome of the symmetry operators. Once this is obtained, the theory of stabilizers allows us to rotate the symmetry generators onto single Pauli $X$ operators, and since they must commute universally every operator of the rotated Hamiltonian will consist of an identity or Pauli $X$ in the corresponding qubit position. This means we can drop the qubit from the Hamiltonian, leaving in its place the eigenvalue determined by the chosen sector.

In [1]:
from symred.symplectic_form import PauliwordOp, QubitHamiltonian, MajoranaOp, convert_openF_fermionic_op_to_maj_op
import numpy as np
from openfermion import MajoranaOperator, FermionOperator, get_majorana_operator, QubitOperator, get_sparse_operator

from openfermion import jordan_wigner, bravyi_kitaev
from openfermion.transforms import reverse_jordan_wigner
import os
import json
from functools import reduce

In [2]:
operator1 = [
    [4],
]
coeffs1 = np.arange(2,len(operator1)+2)
M1 = MajoranaOp(operator1, coeffs1)
print(M1)
print()

operator2 = [
    [2],
    [3]
]
coeffs2 = 1*np.arange(5,len(operator2)+5)
M2 = MajoranaOp(operator2, coeffs2)
print(M2)
print()

M3 = M1 + M2

print(M3)
print()

M4 =M1 * M2
print(M4)
print()

M1_openf = M1.to_OF_op()
M2_openf = M2.to_OF_op()
print(M1_openf+M2_openf)
print()
print(M1_openf*M2_openf)

(2+0j) γ4

(5+0j) γ2 +
(6+0j) γ3

(2+0j) γ4 +
(6+0j) γ3 +
(5+0j) γ2

(-12+0j) γ3 γ4 +
(-10+0j) γ2 γ4

(5+0j) (2,) +
(6+0j) (3,) +
(2+0j) (4,)

(-10+0j) (2, 4) +
(-12+0j) (3, 4)


In [3]:
y1 = MajoranaOperator(term=(4,), coefficient=2)
y2 = MajoranaOperator(term=(2,), coefficient=5) + MajoranaOperator(term=(3,), coefficient=6)
y1*y2

MajoranaOperator.from_dict(terms={(2, 4): -10, (3, 4): -12})

In [4]:
operator = [
    [0,1,2,3],
    [5,6],
    [3,5],
    [0,1,2,3,4,5,6,7],
    [2,3],
    [4]
]

# operator = [
#     [0],
#     [1,2],
#     [3,4],
#     [0,1,2,3,4,5,6,7],
#     [2,3]
# ]

# operator = [
#     [1],
#     [2],
#     [3]

# ]

# # #
# operator = [
#     [0]
# ]


# operator = [
#     [1]
# ]

coeffs = np.arange(2,len(operator)+2)

###
M = MajoranaOp(operator, coeffs)
print(M)

M.adjacency_matrix()

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4


array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

In [5]:
print(M*M)
print()

C = M.to_OF_op()
out = C*C
print(out)
# print()
# print(C)

(17+0j) I +
(42+0j) γ4 γ5 γ6 +
(20+0j) γ4 γ5 γ6 γ7 +
(-56+0j) γ3 γ4 γ5 +
(36+0j) γ2 γ3 γ5 γ6 +
(84+0j) γ2 γ3 γ4 +
(-24+0j) γ0 γ1 +
(-60+0j) γ0 γ1 γ4 γ5 γ6 γ7 +
(40+0j) γ0 γ1 γ2 γ4 γ6 γ7 +
(12+0j) γ0 γ1 γ2 γ3 γ5 γ6 +
(28+0j) γ0 γ1 γ2 γ3 γ4 +
(-30+0j) γ0 γ1 γ2 γ3 γ4 γ7

(17+0j) () +
(-24+0j) (0, 1) +
(28+0j) (0, 1, 2, 3, 4) +
(-30+0j) (0, 1, 2, 3, 4, 7) +
(12+0j) (0, 1, 2, 3, 5, 6) +
(40+0j) (0, 1, 2, 4, 6, 7) +
(-60+0j) (0, 1, 4, 5, 6, 7) +
(84+0j) (2, 3, 4) +
(36+0j) (2, 3, 5, 6) +
(-56+0j) (3, 4, 5) +
(42+0j) (4, 5, 6) +
(20+0j) (4, 5, 6, 7)


In [6]:
M.adjacency_matrix()

array([[0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 1, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]])

# get basis for operator

In [893]:
from symred.S3_projection import gf2_basis_for_gf2_rref, gf2_gaus_elim
ZX_symp = M.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

array([[1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 1, 1, 0]])

In [8]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6


In [9]:
# check basis commutes with orginal op!
print(M)
print()
print(basis_op)

openF_M_op = M.to_OF_op()
basis_op_openF =  basis_op.to_OF_op()
print('commmutes: ', openF_M_op*basis_op_openF == basis_op_openF*openF_M_op)

print(M.commutes_termwise(basis_op))
M.commutes(basis_op)

(2+0j) γ0 γ1 γ2 γ3 +
(3+0j) γ5 γ6 +
(4+0j) γ3 γ5 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7 +
(6+0j) γ2 γ3 +
(7+0j) γ4

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6
commmutes:  True
[[0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]]


True

In [10]:
# next we want to generate pairs according to (2i, 2i+1) == Pauli Z operators!

In [11]:
print(basis_op, '\n')

## valid rots:
# rot = [2,3,4,6]
rot = [2,3,5,7]
op1 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op1_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rotated_basis = op1*basis_op*op1_dag
print(rotated_basis)

op1.commutes_termwise(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6 

-1j γ6 γ7 +
(1+0j) γ0 γ1


array([[0, 0],
       [0, 1]])

In [12]:
jordan_wigner((rotated_basis.to_OF_op()))

1j [Z0] +
(1+0j) [Z3]

In [13]:
rotated_H = op1*M*op1_dag
print(rotated_H)

(7+0j) γ4 +
(4+0j) γ3 γ5 +
(6+0j) γ2 γ3 +
3j γ2 γ3 γ6 γ7 +
(2+0j) γ0 γ1 γ2 γ3 +
(5+0j) γ0 γ1 γ2 γ3 γ4 γ5 γ6 γ7


In [14]:
# should have Z/I op on qubits 0 and 3!!!
jordan_wigner(rotated_H.to_OF_op())

(-2+0j) [Z0 Z1] +
(7+0j) [Z0 Z1 X2] +
(5+0j) [Z0 Z1 Z2 Z3] +
4j [X1 Y2] +
6j [Z1] +
-3j [Z1 Z3]

In [15]:
# check converter from Fermion op to Maj op

In [16]:
# ham = (FermionOperator('0^ 3', .5) +
#        FermionOperator('3^ 0', 0.5) +
#       FermionOperator('3^ 2^ 0 1', 0.5))

# ham = (FermionOperator('0^ 3', .5) +
#        FermionOperator('3^ 0', 0.5) +
#       FermionOperator('3^ 2^ 0 1', 0.5))

ham = (FermionOperator('0^ 3', .5) +
       FermionOperator('3^ 0', 0.5) 
       
#      + FermionOperator('3^ 2^ 0 1', 0.5)
      )

M_out = convert_openF_fermionic_op_to_maj_op(ham)

print('correct: ', M_out.to_OF_op() == get_majorana_operator(ham))

print(M_out)

correct:  True
-0.25j γ1 γ6 +
0.25j γ0 γ7


In [17]:
#mapping to X and Y

In [18]:
print(basis_op, '\n')

## valid rots:
rot = [0]
op1 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op1_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rotated_basis1 = op1*basis_op*op1_dag
print(rotated_basis1)

op1.commutes_termwise(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6 

(1+0j) γ2 γ3 γ5 γ6 +
1j γ1


array([[0, 0],
       [1, 0]])

In [19]:
print(basis_op, '\n')

## valid rots:
# rot = [1,2,3,5,7]
rot = [1,3,5,6,7]
op2 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op2_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rotated_basis2 = op2*rotated_basis1*op2_dag
print(rotated_basis2)

op2.commutes_termwise(rotated_basis1)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6 

1j γ1 +
1j γ1 γ2 γ7


array([[0., 0.],
       [1., 0.]])

In [20]:
from openfermion.transforms.opconversions import get_fermion_operator

XX = get_majorana_operator(ham)
out = get_fermion_operator(XX)
print(out)

(-0.5+0j) [0 3^] +
(0.5+0j) [0^ 3]


In [21]:
# from openfermion import get_sparse_operator

# HH = get_sparse_operator(get_majorana_operator(ham))

# np.linalg.eig(HH.todense())

In [22]:
print(basis_op, '\n')

## valid rots:
# rot = [1,2,3,5,7]
rot = [1,2]
op3 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op3_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rotated_basis3 = op3*rotated_basis2*op3_dag
print(rotated_basis3)

op3.commutes_termwise(rotated_basis2)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6 

(1+0j) γ2 +
1j γ1


array([[0., 0.],
       [1., 0.]])

In [23]:
jordan_wigner((rotated_basis3.to_OF_op()))

1j [Y0] +
(1+0j) [Z0 X1]

In [24]:
p1 = MajoranaOp([[4]],
                [1]) # ZZX

p2 = MajoranaOp([[0,1]],# <- Z0
                [1])
p3 = MajoranaOp([[2,3]], # <- Z1
                [1])
p4 = p1*p2*p3

print(p4)
jordan_wigner((p4.to_OF_op()))


(1+0j) γ0 γ1 γ2 γ3 γ4


(-1+0j) [X2]

In [25]:
p1 = MajoranaOp([[5]], # <--- changed to 5
                [1]) # ZZY

p2 = MajoranaOp([[0,1]], # <- Z0
                [1])
p3 = MajoranaOp([[2,3]], # <- 1
                [1])
p4 = p1*p2*p3

print(p4)
jordan_wigner((p4.to_OF_op()))


(1+0j) γ0 γ1 γ2 γ3 γ5


(-1+0j) [Y2]

In [26]:
p1 = MajoranaOp([[2,3]],
                [1])

jordan_wigner((p1.to_OF_op()))


1j [Z1]

In [27]:
p3 = MajoranaOp([[6,7]],
                [1])

jordan_wigner((p3.to_OF_op()))

1j [Z3]

In [28]:
print(basis_op)

(1+0j) γ0 γ1 +
(1+0j) γ2 γ3 γ5 γ6


In [1282]:
working_dir = os.getcwd()
parent_dir = os.path.dirname(working_dir)
data_dir = os.path.join(parent_dir,'data')



file = 'O1_STO-3G_triplet_OO.json'
# file = 'H2-Be1_STO-3G_singlet_BeH2BeH2.json'

file_path = os.path.join(data_dir, file)
with open(file_path, 'r') as input_file:
    ham_dict = json.load(input_file)
    
ham = QubitHamiltonian(ham_dict)

ham_opemF = ham.PauliwordOp_to_OF

H_qubit = reduce(lambda x,y: x+y, ham_opemF)

fermionic_H = reverse_jordan_wigner(H_qubit)

In [1283]:
maj_H = convert_openF_fermionic_op_to_maj_op(fermionic_H)

In [1571]:
ZX_symp = maj_H.symp_matrix
reduced = gf2_gaus_elim(ZX_symp)
kernel  =  gf2_basis_for_gf2_rref(reduced)

kernel = kernel.astype(int)
kernel

array([[1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]])

In [1578]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [1286]:
basis_op.commutes(basis_op)

True

In [901]:
# print(maj_H.commutes_termwise(basis_op))
maj_H.commutes(basis_op)

True

In [1442]:
jordan_wigner(basis_op.to_OF_op())

1j [Z0 Z3 Z5 Z7 Z9] +
1j [Z1 Z3 Z5 Z7 Z9] +
(-1+0j) [Z2 Z3] +
(-1+0j) [Z4 Z5] +
(1+0j) [Z6 Z7 Z8 Z9]

In [1433]:
# even_inds = np.arange(0,basis_op.n_sites, 2) # 2i positions
# odd_inds = np.arange(1,basis_op.n_sites, 2) # 2i+1 positions

basis_op.symp_matrix[:, basis_op.even_inds]
basis_op.symp_matrix[:, basis_op.odd_inds]

array([[1, 0, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])

In [1288]:
# remember consider pairs of sites!
pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

row_sum = np.einsum('ij->i',pairs)
sqp_indices = np.where(pairs[np.where(row_sum==1)])[1]

non_sqp_basis = MajoranaOp(basis_op.symp_matrix[np.where(row_sum!=1)],
                            basis_op.coeff_vec[np.where(row_sum!=1)])

# _recursive_rotate_onto_sqp(non_sqp_basis)

# rotated_op = self.recursive_rotate_by_Pword(rotations)


In [1324]:
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [1388]:
# jordan_wigner((rott * op1 * rott2).to_OF_op())

In [1398]:
## for non pairs use individual
# op1 = MajoranaOp([[1,6,7,10,11,14,15,18,19]],[1])
# allowed_inds= [0,6,7,10,11,14,15,18,19]

# for pairs used even ind!
op1 = MajoranaOp([[0,1,6,7,10,11,14,15,18,19]],[1])
allowed_inds= [0,6,7,10,11,14,15,18,19]

rott = MajoranaOp([allowed_inds],[1])
phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
phase =(1j)**(sum(np.logical_and(new_op[rott.even_inds],
                                 new_op[rott.odd_inds])))

rott = MajoranaOp([[], allowed_inds] ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp([[], allowed_inds],[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

print(rott.commutes(op1))

print(rott * op1 * rott2)


# rot_op = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])

False
1j γ1


In [1485]:
used_indices=[]

In [1579]:
row_sum = np.einsum('ij->i',basis_op.symp_matrix)
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)
int_list = pairs @ (1 << np.arange(pairs.shape[1])[::-1])

sort_rows_by_weight = np.argsort(-1*int_list)
pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0] # 0th index

non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))

# note pairs[sort_rows_by_weight][0] is term we are reducing too!
even_inds = 2*np.where(np.logical_and(np.einsum('ij->j',pairs)==1, pairs[sort_rows_by_weight][0])==True)[0]
odd_inds = even_inds + 1

even_selected = even_inds[0]
odd_selected = odd_inds[0]

pivot_row_rot = deepcopy(pivot_row.copy())

# if both odd even position 1 then
if np.logical_and(pivot_row_rot[even_selected], pivot_row_rot[odd_selected]):
    # set either odd or even selected index to zero
    pivot_row_rot[even_selected]=0
else:
    # reverse odd even indices! (uses mod 2 addition)
    pivot_row_rot[np.array([even_selected, odd_selected])]= (pivot_row_rot[np.array([even_selected, odd_selected])]+1)%2

phase =(1j)**(sum(np.logical_and(pivot_row_rot[basis_op.even_inds],
                                 pivot_row_rot[basis_op.odd_inds])))

rott = MajoranaOp(np.array([np.zeros_like(pivot_row_rot),
                            pivot_row_rot]) ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot), 
                             pivot_row_rot]),[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

print(rott * basis_op * rott2)

basis_op= rott * basis_op * rott2

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0


In [1550]:
pairs=pairs[non_single_terms]
pairs

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]])

In [1618]:
if kernel.shape[0]:
    basis_coeffs = np.ones(kernel.shape[0])
else:
    basis_coeffs=[1]

basis_op = MajoranaOp(kernel, basis_coeffs)
print(basis_op)

(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19


In [1623]:
row_sum = np.einsum('ij->i',basis_op.symp_matrix)
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)
non_single_terms = np.einsum('ij->i', pairs)>1
if all(np.einsum('ij->i', pairs)==1):
    raise ValueError('done')
pairs=pairs[non_single_terms]

int_list = pairs @ (1 << np.arange(pairs.shape[1])[::-1])

sort_rows_by_weight = np.argsort(-1*int_list)
pivot_row = basis_op.symp_matrix[non_single_terms][sort_rows_by_weight][0] # 0th index

non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))

# note pairs[sort_rows_by_weight][0] is term we are reducing too!
even_inds = 2*np.where(np.logical_and(np.einsum('ij->j',pairs)==1, pairs[sort_rows_by_weight][0])==True)[0]
odd_inds = even_inds + 1

even_selected = even_inds[0]
odd_selected = odd_inds[0]

pivot_row_rot = deepcopy(pivot_row.copy())

# if both odd even position 1 then
if np.logical_and(pivot_row_rot[even_selected], pivot_row_rot[odd_selected]):
    # set either odd or even selected index to zero
    pivot_row_rot[even_selected]=0
    
    rott3 = MajoranaOp([[], [odd_selected]],[np.cos(np.pi/4),  1j*np.sin(np.pi/4)])
    rott4 = MajoranaOp([[], [odd_selected]],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])
else:
    # reverse odd even indices! (uses mod 2 addition)
    pivot_row_rot[np.array([even_selected, odd_selected])]= (pivot_row_rot[np.array([even_selected, odd_selected])]+1)%2
    
    rott3 = MajoranaOp([[]],[1])
    rott4 = MajoranaOp([[]],[1])
    

phase =(1j)**(sum(np.logical_and(pivot_row_rot[basis_op.even_inds],
                                 pivot_row_rot[basis_op.odd_inds])))

rott1 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot),
                            pivot_row_rot]) ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot), 
                             pivot_row_rot]),[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

used_indices.append(even_selected)
used_indices.append(odd_selected)

basis_op= rott1 * basis_op * rott2
basis_op= rott3 * basis_op * rott4
print(basis_op)



1j γ12 γ13 +
-1j γ8 γ9 +
-1j γ4 γ5 +
(-1+0j) γ2 γ3 +
(-1+0j) γ0 γ1


In [1628]:
rev = reverse_jordan_wigner(jordan_wigner(basis_op.to_OF_op()))
bravyi_kitaev(rev)

-1j [Z0] +
-1j [Z0 Z1] +
(1+0j) [Z2] +
(1+0j) [Z4] +
(-1+0j) [Z6]

In [1648]:
# ooo = MajoranaOp([[2,3]],[1])
# ooo = MajoranaOp([[6,7]],[1])
ooo = MajoranaOp([[14,15]],[1])
rev = reverse_jordan_wigner(jordan_wigner(ooo.to_OF_op()))
bravyi_kitaev(rev)

1j [Z3 Z5 Z6 Z7]

In [1651]:
rev = reverse_jordan_wigner(QubitOperator('X0 Z1'))
bravyi_kitaev(rev)

(-1+0j) [Y0 Y1]

In [ ]:
asdf

In [1452]:
while not all(np.einsum('ij->i',pairs)==1):
    
    row_sum = np.einsum('ij->i',basis_op.symp_matrix)
    col_sum = np.einsum('ij->j',basis_op.symp_matrix)

    pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)
    int_list = pairs @ (1 << np.arange(pairs.shape[1])[::-1])

    sort_rows_by_weight = np.argsort(-1*int_list)
    pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0] # 0th index

    non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))

    # note pairs[sort_rows_by_weight][0] is term we are reducing too!
    even_inds = 2*np.where(np.logical_and(np.einsum('ij->j',pairs)==1, pairs[sort_rows_by_weight][0])==True)[0]
    odd_inds = even_inds + 1

    even_selected = even_inds[0]
    odd_selected = odd_inds[0]

    pivot_row_rot = deepcopy(pivot_row.copy())

    # if both odd even position 1 then
    if np.logical_and(pivot_row_rot[even_selected], pivot_row_rot[odd_selected]):
        # set either odd or even selected index to zero
        pivot_row_rot[even_selected]=0
        rott3 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot),
                                pivot_row_rot]) ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
        rott4 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot), 
                                 pivot_row_rot]),[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])
    else:
        # reverse odd even indices! (uses mod 2 addition)
        pivot_row_rot[np.array([even_selected, odd_selected])]= (pivot_row_rot[np.array([even_selected, odd_selected])]+1)%2
        
        rott3 = MajoranaOp([[]],[1])
        rott4 = MajoranaOp([[]],[1])
        
    phase =(1j)**(sum(np.logical_and(pivot_row_rot[basis_op.even_inds],
                                     pivot_row_rot[basis_op.odd_inds])))

    rott = MajoranaOp(np.array([np.zeros_like(pivot_row_rot),
                                pivot_row_rot]) ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    rott2 = MajoranaOp(np.array([np.zeros_like(pivot_row_rot), 
                                 pivot_row_rot]),[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

    print(rott * basis_op * rott2)
    
    basis_op= rott * basis_op * rott2
    pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ

1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12

(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12

-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12

...

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ1

1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12

-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12

(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(-1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ0
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0
(1+0j) γ12

KeyboardInterrupt: 

In [ ]:


non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
support = pivot_row*col_sum
pivot_point_index = non_I_pos[np.argmin(support[non_I_pos])]
# print(pivot_point)

# # pivot_rotation
pivot_row_rot = pivot_row.copy()

pivot_point_index_odd = pivot_point_index + 1

pivot_row_rot[np.array([pivot_point_index,pivot_point_index_odd])] = 1 # set both positions to 1!

used_indices.append(pivot_point_index)
used_indices.append(pivot_point_index_odd)

rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))

rot_maj = MajoranaOp(np.array([pivot_row_rot]), [1])

# pivot_row[pivot_point_index, pivot_point_index_odd] #=1

# rotations.append((symplectic_to_string(base_vector), None))
# used_indices.append(pivot_index_X)
# used_indices.append(pivot_index_X + self.n_qubits)
rotations


In [1294]:
used_indices =[]
rotations=[]

In [1210]:
row_sum = np.einsum('ij->i',basis_op.symp_matrix)
col_sum = np.einsum('ij->j',basis_op.symp_matrix)

pairs = np.logical_or(basis_op.symp_matrix[:, basis_op.even_inds], basis_op.symp_matrix[:, basis_op.odd_inds]).astype(int)
int_list = pairs @ (1 << np.arange(pairs.shape[1])[::-1])

sort_rows_by_weight = np.argsort(-1*int_list)
pivot_row = basis_op.symp_matrix[sort_rows_by_weight][0] # 0th index

non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
support = pivot_row*col_sum
pivot_point_index = non_I_pos[np.argmin(support[non_I_pos])]
# print(pivot_point)

# # pivot_rotation
pivot_row_rot = pivot_row.copy()

pivot_point_index_odd = pivot_point_index + 1

pivot_row_rot[np.array([pivot_point_index,pivot_point_index_odd])] = 1 # set both positions to 1!

used_indices.append(pivot_point_index)
used_indices.append(pivot_point_index_odd)

rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))

rot_maj = MajoranaOp(np.array([pivot_row_rot]), [1])

# pivot_row[pivot_point_index, pivot_point_index_odd] #=1

# rotations.append((symplectic_to_string(base_vector), None))
# used_indices.append(pivot_index_X)
# used_indices.append(pivot_index_X + self.n_qubits)
rotations


[('(1+0j) γ0 γ1 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19', None)]

In [1039]:
print(jordan_wigner(rot_op.to_OF_op()))
print()
print(jordan_wigner(basis_op.to_OF_op()))

(0.7071067811865476+0j) [] +
0.7071067811865475j [Y0 Z3 Z5 Z7 Z9]

1j [Z0 Z3 Z5 Z7 Z9] +
1j [Z1 Z3 Z5 Z7 Z9] +
(-1+0j) [Z2 Z3] +
(-1+0j) [Z4 Z5] +
(1+0j) [Z6 Z7 Z8 Z9]


In [1047]:
new_op = basis_op.symp_matrix[sort_rows_by_weight][0]
new_op[pivot_point_index]=0
new_op[pivot_point_index_odd]=1

new_op_inds = np.where(new_op!=0)[0]

phase =(1j)**(sum(np.logical_and(new_op[basis_op.even_inds],
                                 new_op[basis_op.odd_inds])))

phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
rot_op = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rot_op_conj = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])



rot_basis = rot_op * basis_op * rot_op_conj
print(rot_basis)

# print(basis_op)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0


In [1211]:
new_op = basis_op.symp_matrix[sort_rows_by_weight][0]
# new_op[[0,1]]=0
new_op[1]=0
# new_op[0]=0
new_op_inds = np.where(new_op!=0)[0]

# new_op_inds = np.hstack((np.array([18,19]),new_op_inds))

rot_op = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
rot_op_conj = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot_basis = rot_op * basis_op * rot_op_conj
print(rot_basis)

# print(basis_op)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ1


In [48]:
op1 = MajoranaOp(np.array([basis_op.symp_matrix[sort_rows_by_weight][0]]),[1])

print(op1.commutes(rot_op))
print(rot_op * op1 * rot_op_conj)

True
-1j γ0 γ1


In [56]:
rot_op_inds

array([ 6,  7, 10, 11, 14, 15, 18, 19])

In [100]:
print(basis)
basis.symp_matrix[sort_rows_by_weight][0].copy()

pivot_index_odd

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0


3

In [1212]:
print(rot_basis)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ1


In [1241]:
basis = rot_basis.copy()    

# check if unique (equiv to single qubit Pauli operators!)
mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                           basis.symp_matrix[:, basis.odd_inds]).astype(int)

mode_row_sum = np.einsum('ij->i',mode_pairs)
unique_rows = (mode_row_sum==1)


if all(unique_rows):
    # all unique!
    pass
else:
    # sort basis by left most terms

    unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]

#         even_inds_used = np.sort(2*unqiue_mode_indices)
#         odd_inds_used = even_inds_used+1
#         used_inds = np.union1d(even_inds_used, odd_inds_used)
#         free_inds = np.setdiff1d(np.arange(basis.symp_matrix.shape[1]), used_inds)

    # find rows of mode_pairs matrix that are not already unique
    free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

    ## use integar sort to sort non_unique rows
    int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
    sort_rows_by_weight = np.argsort(-1*int_list)

    ## find indpendent index of pivot! (cannot use dependent term)
    # sum down column... note any term with 1 in row is unique term
    support = np.einsum('ij->j', mode_pairs)
    support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
    pivot_index_even = 2*np.where(support==1)[0][0] # gives independent index for symp matrix (not times 2 as 2i not i)
    pivot_index_odd = pivot_index_even + 1 # get odd pivot index too
    
    # take term we are rotating to single term
    rot_op = basis.symp_matrix[sort_rows_by_weight][0].copy()
    
    print('reduce:', jordan_wigner(MajoranaOp(np.array([rot_op]), [1]).to_OF_op()))
    if rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 1:
        rot_op[pivot_index_odd]=0
    elif rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 0:
        raise ValueError('pivot error')
    
    rot_op[0]=1
    rot_op[1]=1
    
    phase =(1j)**(sum(np.logical_and(rot_op[basis_op.even_inds],
                                     rot_op[basis_op.odd_inds])))

    print(phase)
    
    phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}

    rot_op_inds = np.where(rot_op!=0)[0]
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])
#         rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))
    
    print('using:', jordan_wigner(maj_rot_op.to_OF_op()))
#     print(maj_rot_op)
    rot_basis_out = maj_rot_op * basis * maj_rot_op_conj
    
#     print(maj_rot_op * MajoranaOp(np.array([basis.symp_matrix[sort_rows_by_weight][0]]),[1]) * maj_rot_op_conj)

print()
print(rot_basis_out)
print()
print(jordan_wigner(rot_basis_out.to_OF_op()))


reduce: 1j [Z1 Z3 Z5 Z7 Z9]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [X1 Z3 Z5 Z7 Z9]

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
1j γ1 +
(1+0j) γ0 γ1 γ3

1j [Y0] +
1j [Y1] +
(-1+0j) [Z2 Z3] +
(-1+0j) [Z4 Z5] +
(1+0j) [Z6 Z7 Z8 Z9]


1j [Y0] +
1j [Y1] +
(-1+0j) [Z2 Z3] +
(-1+0j) [Z4 Z5] +
(1+0j) [Z6 Z7 Z8 Z9]

In [1232]:
jordan_wigner(MajoranaOp([[0,1,3,4,5]],[1]).to_OF_op())

(-1+0j) [Y1 Z2]

In [1228]:
jordan_wigner(MajoranaOp([[2,3, 4,5]],[1]).to_OF_op())

(-1+0j) [Z1 Z2]

In [1220]:
print(basis)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ1


In [1111]:
# print(maj_rot_op)
# print(basis)
print(maj_rot_op.commutes_termwise(basis))

print(basis)

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1.]]
(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
-1j γ0


In [690]:
ll = [2,3,6,7,10,11,14,15,18,19]
full = MajoranaOp([ll],[1])
jordan_wigner(full.to_OF_op())


1j [Z1 Z3 Z5 Z7 Z9]

In [681]:
ll = [2,3,6,7,10,11,14,15,18,19]
full = MajoranaOp([ll,
                [1]], [1,1j])
# print(full)
# k = [1,2,6,7,10,11,14,15,18,19]
rot = MajoranaOp([[],
                  k
                 ], [np.cos(np.pi/4), 1j*np.sin(np.pi/4)])

rot_conj = MajoranaOp([[],
                 k
                 ], [np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

print(rot *full* rot_conj)


-1j γ2 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ1 γ3


In [596]:
print(rot_c * z1)
print(z1*rot_c)
print()
print(rot_c * z2367101114151819)
print(z2367101114151819*rot_c)

(1+0j) γ2 γ3 γ6 γ7 γ8 γ11 γ14 γ15 γ18 γ19
(1+0j) γ2 γ3 γ6 γ7 γ8 γ11 γ14 γ15 γ18 γ19

(-1+0j) γ1 γ8 γ10
(1+0j) γ1 γ8 γ10


In [601]:
k = [2,6,7,10,11,14,15,18,19]
rot_c =  MajoranaOp([k],[1])

print(rot_c * z2367101114151819)
print(z2367101114151819*rot_c)

(1+0j) γ3
(-1+0j) γ3


In [668]:
z01 =  MajoranaOp([[0,1]],[1])
z2367101114151819 =  MajoranaOp([[2,3,6,7,10,11,14,15,18,19]],[1])

full = z01 + z2367101114151819

k = [3]
rot_c =  MajoranaOp([k],[1])
print(rot_c.commutes(z01))
print(rot_c.commutes(z2367101114151819))

rot = MajoranaOp([[],
                  k
                 ], [np.cos(np.pi/4), 1j*np.sin(np.pi/4)])

rot_conj = MajoranaOp([[],
                 k
                 ], [np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

print(rot *full* rot_conj)

True
False
-1j γ2 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ0 γ1


In [150]:
print(rot_basis)

(1+0j) γ12 γ13 γ14 γ15 γ16 γ17 γ18 γ19 +
(1+0j) γ8 γ9 γ10 γ11 +
(1+0j) γ4 γ5 γ6 γ7 +
(1+0j) γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
1j γ1


In [ ]:
print(op1 * MajoranaOp([new_op_inds], [1]))

In [ ]:
print(rot_op)

In [ ]:
rot_basis

In [ ]:

row_sum==1

In [ ]:
pairs = np.logical_or(rot_basis.symp_matrix[:, rot_basis.even_inds], 
                      rot_basis.symp_matrix[:, rot_basis.odd_inds]).astype(int)
row_sum = np.einsum('ij->i',pairs)

sqp_indices = np.where(pairs[np.where(row_sum==1)])[1]
even_inds_used = np.sort(2*sqp_indices)
odd_inds_used = even_inds_used+1


row_sum = np.einsum('ij->i',rot_basis.symp_matrix)
col_sum = np.einsum('ij->j',rot_basis.symp_matrix)
int_list = pairs @ (1 << np.arange(pairs.shape[1])[::-1])

sort_rows_by_weight = np.argsort(-1*int_list)
pivot_row = rot_basis.symp_matrix[sort_rows_by_weight][0] # 0th index
non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
support = pivot_row*col_sum
pivot_point_index = non_I_pos[np.argmin(support[non_I])]

In [ ]:

# check if unique (equiv to single qubit Pauli operators!)
mode_pairs = np.logical_or(rot_basis.symp_matrix[:, rot_basis.even_inds], 
                           rot_basis.symp_matrix[:, rot_basis.odd_inds]).astype(int)


    
mode_row_sum = np.einsum('ij->i',mode_pairs)
unique_rows = (mode_row_sum==1)

unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]

# even_inds_used = np.sort(2*unqiue_mode_indices)
# odd_inds_used = even_inds_used+1
# used_inds = np.union1d(even_inds_used, odd_inds_used)

# free_inds = np.setdiff1d(np.arange(rot_basis.symp_matrix.shape[1]), used_inds)
# print(rot_basis.symp_matrix[:,free_inds])

# mode_pairs
free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

## use integar sort
int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
sort_rows_by_weight = np.argsort(-1*int_list)

# take left most index (0th index)



## find indpendent index of pivot! (cannot use dependent term)
# sum down column... note any term with 1 in row is unique term
support = np.einsum('ij->j', mode_pairs)
support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
pivot_ind = np.where(support==1)[0][0] # gives independent index

pivot_row = rot_basis.symp_matrix[sort_rows_by_weight][0]
print(pivot_row[2*pivot_ind])



In [ ]:
mode_row_sum = np.einsum('ij->i',mode_pairs)
unqiue_mode_indices = np.where(mode_pairs[np.where(mode_row_sum==1)])[1]

even_inds_used = np.sort(2*unqiue_mode_indices)
odd_inds_used = even_inds_used+1

even_inds_used

In [ ]:
XX = np.array([[0,1,1,1,1],
               [0,1,0,1,0]])
row_sum = np.einsum('ij->i',XX)
col_sum = np.einsum('ij->j',XX)

int_list = XX @ (1 << np.arange(XX.shape[1])[::-1])

sort_rows_by_weight = np.argsort(-1*int_list)
pivot_row = XX[sort_rows_by_weight][0] # 0th index
# non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
# non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
# support = pivot_row*col_sum
# pivot_point_index = non_I_pos[np.argmin(support[non_I])]

# non_I_pos = np.setdiff1d(np.where(pivot_row)[0], np.array([]))
# support = pivot_row*col_sum
# pivot_point_index = non_I_pos[np.argmin(support[non_I_pos])]
# pivot_point_index

# non_I_pos[np.argmin(np.einsum('ij->j', XX)[non_I_pos])]

support = np.einsum('ij->j', XX)
np.where(support==1)[0][0]

In [ ]:
fasdf

In [ ]:
def recursively_rotated_single_maj(basis_op):
    
    basis = basis_op.copy()    
    
    # check if unique (equiv to single qubit Pauli operators!)
    mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                               basis.symp_matrix[:, basis.odd_inds]).astype(int)
    
    mode_row_sum = np.einsum('ij->i',mode_pairs)
    unique_rows = (mode_row_sum==1)
    
    
    if all(unique_rows):
        # all unique!
        return basis
    else:
        # sort basis by left most terms
    
        unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]
        
#         even_inds_used = np.sort(2*unqiue_mode_indices)
#         odd_inds_used = even_inds_used+1
#         used_inds = np.union1d(even_inds_used, odd_inds_used)
#         free_inds = np.setdiff1d(np.arange(basis.symp_matrix.shape[1]), used_inds)
        
        # find rows of mode_pairs matrix that are not already unique
        free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

        ## use integar sort to sort non_unique rows
        int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
        sort_rows_by_weight = np.argsort(-1*int_list)
        
        ## find indpendent index of pivot! (cannot use dependent term)
        # sum down column... note any term with 1 in row is unique term
        support = np.einsum('ij->j', mode_pairs)
        support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
        pivot_index_even = 2*np.where(support==1)[0][0] # gives independent index for symp matrix (not times 2 as 2i not i)
        pivot_index_odd = pivot_index_even + 1 # get odd pivot index too
        
#         # take term we are rotating to single term
#         rot_op = basis.symp_matrix[sort_rows_by_weight][0].copy()
#         if rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 1:
#             rot_op[pivot_index_odd]=0
#         elif rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 0:
#             raise ValueError('pivot error')
#         rot_op_inds = np.where(new_op!=0)[0]

        

        maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
        maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1j*np.sin(np.pi/4)])
#         rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))

        rot_basis = maj_rot_op * basis * maj_rot_op_conj
        print(maj_rot_op * MajoranaOp(np.array([basis.symp_matrix[sort_rows_by_weight][0]]),[1]) * maj_rot_op_conj)
        return recursively_rotated_single_maj(rot_basis)
    

In [ ]:
new_op = basis_op.symp_matrix[sort_rows_by_weight][0]
new_op[[0,1]]=0
# new_op[1]=0
# new_op[0]=0
new_op_inds = np.where(new_op!=0)[0]

new_op_inds = np.hstack((np.array([18,19]),new_op_inds))

rot_op = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
rot_op_conj = MajoranaOp([[],new_op_inds], [np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot_basis = rot_op * basis_op * rot_op_conj
print(rot_basis)

# print(basis_op)

In [ ]:
def _recursive_rotate_onto_sqp(basis: StabilizerOp):
    """ recursively generates Clifford operations mapping the input basis 
    onto single-qubit Pauli operators. Works in order of increasing term
    weight (i.e. the number of non-identity positions)
    """
    if basis is None:
        return None
    else:
        row_sum = np.einsum('ij->i',basis.symp_matrix)
        col_sum = np.einsum('ij->j',basis.symp_matrix)
        sort_rows_by_weight = np.argsort(row_sum)
        pivot_row = basis.symp_matrix[sort_rows_by_weight][0]
        non_I = np.setdiff1d(np.where(pivot_row)[0], np.array(used_indices))
        support = pivot_row*col_sum
        pivot_point = non_I[np.argmin(support[non_I])]
        pivot_rotation = update_sets(pivot_row.copy(), pivot_point)
        rotated_basis = basis._rotate_by_single_Pword(pivot_rotation)
        non_sqp = np.where(np.einsum('ij->i', rotated_basis.symp_matrix)!=1)[0].tolist()
        try:
            new_basis = reduce(lambda x,y:x+y, [rotated_basis[i] for i in non_sqp])
        except:
            new_basis = None
        return _recursive_rotate_onto_sqp(new_basis)
    
    def update_sets(base_vector, pivot_index):
        """ 
        - ammend the X_, Z_block positions at pivot_index to 1
            (corresponds with fixing the pivot_index qubit to Pauli Y)
        - append the rotation to the rotations list
        - update used_indices with the fixed qubit position.
        - also returns the Pauli rotation so it may be applied in _recursive_rotate_onto_sqp
        """
        pivot_index_X = pivot_index % self.n_qubits # index in the X block
        base_vector[np.array([pivot_index_X, pivot_index_X+self.n_qubits])]=1

        rotations.append((symplectic_to_string(base_vector), None))
        used_indices.append(pivot_index_X)
        used_indices.append(pivot_index_X + self.n_qubits)

        return PauliwordOp(base_vector, [1])

In [ ]:
# identify any basis elements that already single-qubit Paulis 
row_sum = np.einsum('ij->i',self.symp_matrix)
sqp_indices = np.where(self.symp_matrix[np.where(row_sum==1)])[1]
sqp_X_block = sqp_indices % self.n_qubits
used_indices = list(np.concatenate([sqp_X_block, sqp_X_block+self.n_qubits]))
# find rotations for the non-single-qubit Pauli terms
non_sqp_basis = StabilizerOp(self.symp_matrix[np.where(row_sum!=1)],
                            self.coeff_vec[np.where(row_sum!=1)])
if non_sqp_basis.n_terms != 0:
    # i.e. the operator does not already consist of single-qubit Paulis
    _recursive_rotate_onto_sqp(non_sqp_basis)
    rotated_op = self.recursive_rotate_by_Pword(rotations)
else:
    rotated_op = self

In [ ]:
print(basis_op, '\n')

# ## valid rots:
rot = [1,6,7,10,11,14,15,18,19]
# rot = [2,3,5,7]
op1 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op1_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot1 = op1*basis_op*op1_dag
print(rot1)

In [ ]:
print(rot1, '\n')

# ## valid rots:
rot = [1, 13,14,15,16,17,14,18,19]
# rot = [2,3,5,7]
op2 = MajoranaOp([[],rot],[np.cos(np.pi/4), 1j*np.sin(np.pi/4)])
op2_dag = MajoranaOp([[],rot],[np.cos(np.pi/4), -1j*np.sin(np.pi/4)])

rot2 = op2*rot1*op2_dag
print(rot2)


op2.commutes_termwise(rot1)

In [ ]:
out = get_fermion_operator(basis_op.to_OF_op())


# new = convert_openF_fermionic_op_to_maj_op(out)

In [ ]:
op_test = MajoranaOp([[0,1]],[-1j])
print(op_test)

In [ ]:
jordan_wigner(op_test.to_OF_op())

In [ ]:
ferm = get_fermion_operator(op_test.to_OF_op())

mat = get_sparse_operator(ferm).todense()

np.linalg.eigh(mat)

In [ ]:
reverse_jordan_wigner(QubitOperator('X0', 1))

In [ ]:
jordan_wigner(ferm)

In [ ]:
def convert_to_qubit_state(L):
    
    conv = {0: np.array([[1],[0]]), 1: np.array([[0],[1]])}
    
    state = reduce(np.kron, [conv[occ] for occ in L])
    
    return state


def convert_state_to_binary(state):
    
    w = int(np.log2(state.shape[0]))
    state_str=''
    for ind, amp in enumerate(state):

        if abs(amp)>0:
            string = np.binary_repr(ind, width=w)
            state_str += f'({amp})|{string}> \n'

#     print(state_str)
    return state_str
        
        


In [ ]:
x = convert_to_qubit_state([1,0,1])

crea = FermionOperator('1^',-1)

OP = np.kron(get_sparse_operator(crea).todense(), np.eye(2))
new = OP @ x

out = convert_state_to_binary(new)
print(out)

In [ ]:
x = convert_to_qubit_state([1,0,1])

s = convert_state_to_binary(x)
print(s)

In [ ]:
# TODO:
## find how to get operators to map to single and doubles y{i}
## then compare JW and BK transforms!

In [ ]:
from openfermion import hermitian_conjugated

qq = FermionOperator('1^ 1',2)
print(qq == hermitian_conjugated(qq))



In [ ]:
qq = FermionOperator('3^',2) 
#       +
#        FermionOperator('3^ 0', 0.5) +
#       FermionOperator('3^ 2^ 0 1', 0.5))

l = convert_openF_fermionic_op_to_maj_op(qq)
print(l)

In [ ]:
reverse_jordan_wigner(QubitOperator('Z0'))

In [ ]:
get_sparse_operator(FermionOperator('0^',1) * FermionOperator('0',1)).todense()

In [ ]:
get_fermion_operator(MajoranaOperator(term=(0,1)))

In [ ]:
jordan_wigner(get_fermion_operator(MajoranaOperator(term=(0,1))))

In [ ]:
# from symred.S3_projection import QubitTapering

In [ ]:
# # ZZ = PauliwordOp(np.array([0,0,1,1]), [1])

# # taper_hamiltonian = QubitTapering(ZZ)

# # print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
# # print('The symmetry generators are\n')
# # print(taper_hamiltonian.symmetry_generators)
# # print('\nand are rotated onto the single-qubit Pauli operators\n')
# # print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
# # print('\nvia a sequence of Clifford pi/2 rotations\n')
# # print(taper_hamiltonian.stabilizers.stabilizer_rotations[0])

# XXX_YYY = PauliwordOp(np.array([[1,1,1,1,1,1],
#                                [1,1,1,0,0,0]]), [1,1])

# taper_hamiltonian = QubitTapering(XXX_YYY)

# print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
# print('The symmetry generators are\n')
# print(taper_hamiltonian.symmetry_generators)
# print('\nand are rotated onto the single-qubit Pauli operators\n')
# print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
# print('\nvia a sequence of Clifford pi/2 rotations\n')
# # print(taper_hamiltonian.stabilizers.stabilizer_rotations[0])

In [239]:
p1 = MajoranaOp([[1]],[1])
p2 = MajoranaOp([[2]],[1])
p23 = MajoranaOp([[2,3]],[1])

print(p1.commutes(p2))
print(p1.commutes(p23))

False
True


In [862]:
# qubit picture and majorana picture

In [796]:
from openfermion import hermitian_conjugated

oo1 = QubitOperator('Z1 Z3 Z5 Z7 Z9 ')+ QubitOperator('Y0 ')
# rr1 = QubitOperator('', np.cos(np.pi/4)) + 1j * QubitOperator('X1', np.sin(np.pi/4)) 
rr1 = QubitOperator('', np.cos(np.pi/4)) + 1j * QubitOperator('X1  Z3 Z5 Z7 Z9', np.sin(np.pi/4)) 

rr1 * oo1 * hermitian_conjugated(rr1)

(1+0j) [Y0] +
(1+0j) [Y1]

In [798]:
ZZZ = [2,3,6,7,10,11,14,15,18,19]
Y0 = [1]
full = MajoranaOp([ZZZ,
                  Y0],[-1j,1])

jordan_wigner(full.to_OF_op())


(1+0j) [Y0] +
(1+0j) [Z1 Z3 Z5 Z7 Z9]

In [857]:
jordan_wigner(rot_conj.to_OF_op())

(0.7071067811865476+0j) [] +
-0.7071067811865475j [X1 Z3 Z5 Z7 Z9]

In [874]:
# note we don't have any imaginary part to when multiplying sin!
# this is becuase majorana op has this sign!!!

indices = [0,1,2, 6,7, 10,11, 14,15 , 18,19]
rot = MajoranaOp([[],indices], [np.cos(np.pi/4), 1*np.sin(np.pi/4)])
rot_conj = MajoranaOp([[],indices], [np.cos(np.pi/4), -1*np.sin(np.pi/4)])

In [875]:
out = rot * full *rot_conj

jordan_wigner(out.to_OF_op())

(1+0j) [Y0] +
(1+0j) [Y1]

In [878]:
print(full)

-1j γ2 γ3 γ6 γ7 γ10 γ11 γ14 γ15 γ18 γ19 +
(1+0j) γ1


In [885]:
# notices phase!!!
indices = [0,1,2, 6,7, 10,11, 14,15 , 18,19]
kk = MajoranaOp([indices], [1])
jordan_wigner(kk.to_OF_op())

1j [X1 Z3 Z5 Z7 Z9]

In [890]:
rot.commutes_termwise(full)

array([[0, 0],
       [1, 0]])

In [1016]:
gg = MajoranaOp([[0,1]],[1])

# count the number of i occuring (aka pairs of terms generate 1i phase!!!)
# these effect the pauli rotations!!
print((1j)**(sum(np.logical_and(gg.symp_matrix[0, gg.even_inds], gg.symp_matrix[0, gg.odd_inds]))))

jordan_wigner(gg.to_OF_op())

1j


1j [Z0]

In [1014]:
gg = MajoranaOp([[3]],[1])
print((1j)**(sum(np.logical_and(gg.symp_matrix[0, gg.even_inds], gg.symp_matrix[0, gg.odd_inds]))))

jordan_wigner(gg.to_OF_op())

(1+0j)


(1+0j) [Z0 Y1]

In [1015]:
gg = MajoranaOp([[0,1,2,3,5]],[1])
print((1j)**(sum(np.logical_and(gg.symp_matrix[0, gg.even_inds], gg.symp_matrix[0, gg.odd_inds]))))

jordan_wigner(gg.to_OF_op())

(-1+0j)


(-1+0j) [Y2]

In [1030]:
phase =(1j)**(sum(np.logical_and(gg.symp_matrix[0, gg.even_inds], gg.symp_matrix[0, gg.odd_inds])))

# use dict to map to correct phase!
phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
phase_conv[phase]

1

In [1242]:
basis = rot_basis.copy()    



while not all(unique_rows):
    # sort basis by left most terms

    unqiue_mode_indices = np.where(np.einsum('ij->j',mode_pairs[unique_rows])==1)[0]

#         even_inds_used = np.sort(2*unqiue_mode_indices)
#         odd_inds_used = even_inds_used+1
#         used_inds = np.union1d(even_inds_used, odd_inds_used)
#         free_inds = np.setdiff1d(np.arange(basis.symp_matrix.shape[1]), used_inds)

    # find rows of mode_pairs matrix that are not already unique
    free_mode_inds = np.setdiff1d(np.arange(mode_pairs.shape[1]), unqiue_mode_indices)

    ## use integar sort to sort non_unique rows
    int_list = mode_pairs[:, free_mode_inds] @ (1 << np.arange(mode_pairs[:, free_mode_inds].shape[1])[::-1])
    sort_rows_by_weight = np.argsort(-1*int_list)

    ## find indpendent index of pivot! (cannot use dependent term)
    # sum down column... note any term with 1 in row is unique term
    support = np.einsum('ij->j', mode_pairs)
    support[unqiue_mode_indices]=2 # push any unique index to above 1 (so as not to use)
    pivot_index_even = 2*np.where(support==1)[0][0] # gives independent index for symp matrix (not times 2 as 2i not i)
    pivot_index_odd = pivot_index_even + 1 # get odd pivot index too
    
    # take term we are rotating to single term
    rot_op = basis.symp_matrix[sort_rows_by_weight][0].copy()
    
    print('reduce:', jordan_wigner(MajoranaOp(np.array([rot_op]), [1]).to_OF_op()))
    if rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 1:
        rot_op[pivot_index_odd]=0
    elif rot_op[pivot_index_even] == rot_op[pivot_index_odd] == 0:
        raise ValueError('pivot error')
    
    rot_op[0]=1
    rot_op[1]=1
    
    phase =(1j)**(sum(np.logical_and(rot_op[basis_op.even_inds],
                                     rot_op[basis_op.odd_inds])))

    print(phase)
    
    phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}

    rot_op_inds = np.where(rot_op!=0)[0]
    maj_rot_op = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
    maj_rot_op_conj = MajoranaOp([[],rot_op_inds], [np.cos(np.pi/4), -1* phase_conv[phase]* np.sin(np.pi/4)])
#         rotations.append((str(MajoranaOp(np.array([pivot_row_rot]), [1]).__str__()), None))
    
    print('using:', jordan_wigner(maj_rot_op.to_OF_op()))
#     print(maj_rot_op)
    rot_basis_out = maj_rot_op * basis * maj_rot_op_conj
    
    basis = rot_basis_out
    mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                               basis.symp_matrix[:, basis.odd_inds]).astype(int)

    mode_row_sum = np.einsum('ij->i',mode_pairs)
    unique_rows = (mode_row_sum==1)

print()
print(rot_basis_out)
print()
print(jordan_wigner(rot_basis_out.to_OF_op()))


reduce: 1j [Z1 Z3 Z5 Z7 Z9]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [X1 Z3 Z5 Z7 Z9]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]


reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

...

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.7071067811865476+0j) [] +
0.7071067811865475j [Y1]
reduce: 1j [Y1]
1j
using: (0.70710678118

KeyboardInterrupt: 

In [1249]:
mode_pairs = np.logical_or(basis.symp_matrix[:, basis.even_inds], 
                           basis.symp_matrix[:, basis.odd_inds]).astype(int)
mode_pairs

array([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 0],
       [0, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [1271]:

# ma = MajoranaOp([[0,1,3]],[1])
# ma = MajoranaOp([[0,1,2,3,4,5,7]],[1])

ma = MajoranaOp([[8,9]],[1])

jordan_wigner(ma.to_OF_op())

print(jordan_wigner(ma.to_OF_op()))

1j [Z4]


In [1607]:
## for non pairs use individual
# op1 = MajoranaOp([[1,6,7,10,11,14,15,18,19]],[1])
# allowed_inds= [0,6,7,10,11,14,15,18,19]

# for pairs used even ind!
# op1 = MajoranaOp([[0,1,6,7,10,11,14,15,18,19]],[1])
op1 = MajoranaOp([[0,1,6,7,10,11,14,15,18,19], [3,4]],[1,1])
allowed_inds= [0,6,7,10,11,14,15,18,19]

rott = MajoranaOp([allowed_inds],[1])
phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
phase =(1j)**(sum(np.logical_and(new_op[rott.even_inds],
                                 new_op[rott.odd_inds])))

rott = MajoranaOp([[], allowed_inds] ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp([[], allowed_inds],[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])

print(rott.commutes(op1))

print(rott * op1 * rott2)
op1 = rott * op1 * rott2

allowed_inds= [0]

rott = MajoranaOp([allowed_inds],[1])
phase_conv = {1: 1j, -1: -1j, 1j:1, -1j:-1}
phase =(1j)**(sum(np.logical_and(new_op[rott.even_inds],
                                 new_op[rott.odd_inds])))

rott = MajoranaOp([[], allowed_inds] ,[np.cos(np.pi/4), phase_conv[phase]*np.sin(np.pi/4)])
rott2 = MajoranaOp([[], allowed_inds],[np.cos(np.pi/4), -1*phase_conv[phase]*np.sin(np.pi/4)])
print(rott * op1 * rott2)

False
(1+0j) γ3 γ4 +
1j γ1
(1+0j) γ3 γ4 +
(-1+0j) γ0 γ1
